In [ ]:
import jieba
import helper
import pandas as pd
import numpy as np
import re

In [ ]:
baidu_AK = 
baidu_SK = 

In [ ]:
import urllib,sys
import ssl

# client_id 为官网获取的AK， client_secret 为官网获取的SK
host = 'https://aip.baidubce.com/oauth/2.0/token?grant_type=client_credentials&client_id=%s&client_secret=%s' %(baidu_AK, baidu_SK)
print(host)
request = urllib.request.Request(host)
request.add_header('Content-Type', 'application/json; charset=UTF-8')
response = urllib.request.urlopen(request)
content = response.read()
if (content):
    print(content)

In [ ]:
jieba.load_userdict("user_dict.txt")

def preprocessText(x):
    #@todo regex to handle ..., ???, etc
    x = re.sub(r'(。(。*)。)', ' threePeriodsMark ', x)
    x = re.sub(r'(\?(\?*)\?)|(？(？*)？)', " threeQuestionsMark ", x)
    x = re.sub(r'(啊(\啊*)啊)', " threeAHsMark ", x)
    x = re.sub(r'(嗷(\嗷*)嗷)', " threeAOsMark ", x)
    x = re.sub(r'(嗯(\嗯*)嗯)', " threeENsMark ", x)
    x = re.sub(r'(哈(\哈*)哈)', " threeHAsMark ", x)
    x = re.sub(r'(喔(\喔*)喔)', " threeWOsMark ", x)
    x = x.replace(">_<"," emojiMengbi ")
    x = re.sub(r"(。|，|\,)"," ",x)
    return x

def cut_word(x):
    p_x = preprocessText(x)
#     print (p_x)
    seg_list = [word for word in jieba.cut(p_x, cut_all=False) if word != " "]
    return seg_list

print(cut_word("我爱北京天安门"))
print(cut_word("you are my star"))
print(cut_word("我不会java"))
print(cut_word(""))
print(cut_word("我们中出了，一个叛徒"))
print(cut_word("。。。"))
print(cut_word("a。。。。b"))
print(cut_word("。。"))
print(cut_word("..."))
print(cut_word("?"))
print(cut_word("我们."))
print(cut_word("嗯哼"))
print(cut_word("啊嘞????"))
print(cut_word("我擦嘞"))
print(cut_word("我嗷嗷"))
print(cut_word("哈哈"))
print(cut_word("我去"))
print(cut_word("我去北京上学"))
print(cut_word("我去北京上学, nnsd"))
print(cut_word("我靠"))
print(cut_word(">_<"))

In [ ]:
def preprocessing(df):
    df["text_cut"] = df["text"].apply(lambda x: cut_word(x) if not pd.isnull(x) else [])
    return df

In [ ]:
df = helper.load_data(small=False)
df = preprocessing(df)
# df.to_csv("fullData_cutted.csv", index=False)

In [ ]:
df = helper.load_data(small=True)
df = preprocessing(df)
# df.to_csv("smallData_cutted.csv", index=False)

In [ ]:
df.columns

In [ ]:
def unnestCuttedWord(df):
    df["text_cut_num"] = df["text_cut"].apply(lambda x: len(x) if x else 0)
    idColumn = df["id"].repeat(df["text_cut_num"])
    wordColumn = np.concatenate(df["text_cut"].values)
    return pd.DataFrame({"id": idColumn, "word": wordColumn})
    
wordLevelDf = pd.merge(unnestCuttedWord(df), df[["id","from"]], on="id")

In [ ]:
wordLevelDf.to_csv("small_word_level.csv", index=False)

In [ ]:
import json
import requests
token = json.loads(content)["access_token"]
headers = {'content-type' : 'application/json'}
baidu_sentiment_analysis_url = "https://aip.baidubce.com/rpc/2.0/nlp/v1/sentiment_classify"
params = {
    "access_token":token
}

def getSentiment(text):
    post_data = {
       "text": text
    }
    
    if text is None:
        return None
    
    try:
        response = requests.post(baidu_sentiment_analysis_url, params=params, data=json.dumps(post_data), headers=headers)
        if response.status_code == 200:
            res = json.loads(response.text)["items"][0]
            return res
        else:
            return None
    except:
        return None

In [ ]:
cleanDf = df[["id","from","date","text"]]
cleanDf.shape

In [ ]:
sentimentRes = cleanDf.apply(lambda x: getSentiment(x["text"]), axis = 1).apply(pd.Series)

In [ ]:
pd.concat([cleanDf, sentimentRes], axis = 1)